# Init

In [ ]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from PIL import Image, ImageTk, ImageColor
from skimage import filters, segmentation, io, img_as_float, color
from skimage.measure import label, regionprops
from skimage.filters import sobel, apply_hysteresis_threshold


# Data

In [ ]:
# file_path = "../data/Marked 2nd.tif"
file_path = "../data/A1p150_R1,R2_induction_both.tif"
image_raw = Image.fromarray(
    io.imread(file_path)
)
image_raw

# Segmentation

## Individual gels - based on alpha

In [ ]:
image_raw = img_as_float(image_raw)
alpha_label_image = label(image_raw[:, :, 3])
plt.imshow(alpha_label_image)

In [ ]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to console handler
console_handler.setFormatter(formatter)

# add console handler to logger
logger.addHandler(console_handler)

In [ ]:
color_map = ImageColor.colormap
color_map['gray'] = (128, 128, 128)
color_map['red'] = (255, 0, 0)
color_map['green'] = (0, 255, 0)
color_map['blue'] = (0, 0, 255)

gel_images = []

for region in regionprops(alpha_label_image):    
    if region.area >= 25000:
        fig,ax = plt.subplots(
            nrows=4
        )

        gel_minr, gel_minc, gel_maxr, gel_maxc = region.bbox
        gel_image = image_raw[gel_minr:gel_maxr,gel_minc:gel_maxc]
        gel_images.append(gel_image)

        ax[0].imshow(gel_image)

        # grayscale_image = gel_image[:, :, 1]
        grayscale_image = color.rgb2gray(
            gel_image[:, :, :3]
        )

        ax[1].imshow(grayscale_image)
        threshold_value = np.median(grayscale_image) / 1.2
        # threshold_value = filters.threshold_otsu(grayscale_image)
        binary_image = grayscale_image > threshold_value
        ax[2].imshow(binary_image)

        label_image = label(binary_image)

        color_image = Image.fromarray(
            label_image.astype('uint8'),
            mode='P'
        )
        color_image.putpalette(
            sum(
                [
                    list(color_map[name])
                    for name
                    in ('gray', 'red', 'green', 'blue')
                ],
                []
            )
        )
        ax[3].imshow(label_image)
        
        for blot_region in regionprops(label_image):
            if blot_region.area < 100: pass
            
            blot_minr, blot_minc, blot_maxr, blot_maxc = blot_region.bbox
            logger.debug(f'{region.area} VS {blot_region.area}')
            logger.debug(f'{blot_minr}, {blot_minc}, {blot_maxr}, {blot_maxc}')

            # sub_region_image = gel_image[blot_minr:blot_maxr,blot_minc:blot_maxc]
            rect = patches.Rectangle(
                (blot_minc, blot_minr),
                blot_maxc-blot_minc,
                blot_maxr-blot_minr,
                linewidth=5,
                edgecolor='r',
                facecolor='none'
            )
            ax[2].add_patch(rect)

        fig.show()        

In [ ]:
import matplotlib.patches as patches

In [ ]:
fig, ax = plt.subplots(nrows=1)
ax.imshow(label_image)
rect = patches.Rectangle(
    (blot_minc, blot_minr),
    blot_maxc-blot_minc,
    blot_maxr-blot_minr,
    linewidth=5,
    edgecolor='r',
    facecolor='none'
)
ax.add_patch(rect)
fig.show()


In [ ]:


plt.imshow(color_image)
